In [1]:
import numpy as np
from tqdm import tqdm
import random
import gymnasium as gym
# from gym.wrappers import Monitor
import glob
import io
import matplotlib.pyplot as plt
from IPython.display import HTML
from collections import deque

In [2]:
#Setting up the environment
env = gym.make('Taxi-v3', render_mode='ansi')
state, _ = env.reset()

# The state of the environment
print(state)

print("decoded state")
print(list(env.decode(state)))


#The number of states in the environment
print(env.observation_space.n)

#The number of actions in the environment
print(env.action_space.n)

#Take a step in the environment
next_state, reward, done, _, _ = env.step(1)
print("decoded state")
print(list(env.decode(next_state)))

#Render the environment
env.render()

107
decoded state
[1, 0, 1, 3]
500
6
decoded state
[0, 0, 1, 3]


/home/adi/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.decode to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.decode` for environment variables or `env.get_wrapper_attr('decode')` that will search the reminding wrappers.
  logger.warn(


'+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | : |\n|Y| : |\x1bB\x1b: |\n+---------+\n  (North)\n'

[![Options](https://www.researchgate.net/publication/292208109/figure/fig2/AS:960497125978114@1606011768506/Taxi-problem-and-an-action-hierarchy.gif)]

In [3]:
# actions:  ['down', 'up', 'right', 'left', 'pickup', 'dropoff']
actions = [0, 1, 2, 3, 4, 5]


goal_states = [[0, 0], [0, 4], [4, 0], [4, 3]]

level_1_options = [1, 2]
# highest level options: ['get', 'put']

get_options = [3, 4, 5, 6, 7]
# get options: ['pickup', 'getR', 'getG', 'getY', 'getB']

put_options = [3, 4, 5, 6, 7]
# put options: ['dropoff', 'putR', 'putG', 'putY', 'putB']

navigate_options = [8, 9, 10, 11]
# navigate options: ['down', 'up', 'right', 'left']

Q_values = np.zeros((env.observation_space.n, len(level_1_options))) # Q-values for level 1 options -> get, put

Q_level_two_options = np.zeros((2, env.observation_space.n, len(get_options))) # Q-values for level options -> pickup/dropoff, getR/getG/getY/getB

Q_level_three_options = np.zeros((4, env.observation_space.n, len(navigate_options))) # Q-values for level options -> down, up, right, left


# Softmax function
def softmax(Q, state, tau):
    q_values = Q[state]
    q_values = q_values / tau
    max_q = np.max(q_values)
    e = np.exp(q_values - max_q)
    dist = e / np.sum(e)
    action = np.random.choice(len(dist), p=dist)
    return action

def epsilon_greedy(Q, state, epsilon):
    if random.uniform(0, 1) < epsilon:
        action = np.random.randint(0, 4)
    else:
        action = np.argmax(Q[state])
    return action

In [ ]:
def decode_state(env, state):
    return list(env.decode(state))

In [ ]:
### HYPERPARAMETERS ###
ALPHA_Q = 0.1
ALPHA_OPTIONS = 0.3
GAMMA = 0.9
TAU = 0.7
DECAY_CONSTANT = 0.995

In [ ]:
def execute_option(env, Q_values, Q_options, Q_freq, option_freq, option, state, update_Q=True, policy = softmax):
    pass

In [ ]:
#### SMDP Q-Learning
episodes = 10000

main_update_freq = np.zeros(Q_values.shape)
option_update_freq = np.zeros(Q_options.shape)


def SMDP_Q_Learning(env, Q_values, Q_options, main_update_freq, option_update_freq, episodes, policy=softmax, update_Q = True,TAU = TAU):
    # Iterate over 1000 episodes
    total_rewards = np.zeros((episodes))
    for i in tqdm(range(episodes)):
        state, _ = env.reset()    
        done = False
        # While episode is not over
        while not done:
            # Choose action        
            action = policy(Q_values, state, TAU)
            # Execute action
            next_state, reward, done, value_array = execute_option(env, Q_values, Q_options, main_update_freq, option_update_freq, action, state, update_Q=update_Q, policy=policy)
            Q_values, Q_options, main_update_freq, option_update_freq = value_array
            state = next_state
            total_rewards[i] += reward
        TAU = max(0.01, TAU * DECAY_CONSTANT)
    return Q_values, Q_options, main_update_freq, option_update_freq, total_rewards
